<a href="https://colab.research.google.com/github/cya900/-2-/blob/main/%E6%AD%A1%E8%BF%8E%E4%BD%BF%E7%94%A8_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests

In [2]:
import os
import time
import sqlite3
import requests
import hashlib
import base64
import random
import string
from datetime import datetime
from typing import List, Dict, Optional

print("✅ 所有模組導入成功！")

✅ 所有模組導入成功！


In [3]:
#### CONFIG: 編輯這裡來修改設定 ####
CONFIG = {
    # API 模式: "real" 使用真實 RIGSEC API，"mock" 使用模擬資料（測試用）
    "MODE": "mock",  # 改為 "real" 使用真實 API

    # RIGSEC API 設定（僅在 MODE == "real" 時使用）
    "API_BASE": "https://hsm-wallet-xxx-xxx.com",
    "API_KEY": "YOUR_API_KEY",
    "CLIENT_ID": "api_user",
    "CLIENT_SECRET": "client_secret",
    "API_USERNAME": "your_email@example.com",

    # 錢包資訊（真實 API 需要 walletId）
    "WALLET_ID": "111",
    "BLOCKCHAIN": "ethereum",
    "SYMBOL": "ETH",

    # SQLite 資料庫檔案
    "DB_FILE": "addresses.db",

    # 輪詢間隔（秒）
    "POLL_INTERVAL": 10,
}

print(f"✅ 配置載入完成！模式: {CONFIG['MODE']}")


✅ 配置載入完成！模式: mock


In [4]:
#### SQLite 資料庫函數 ####

def init_db(db_file: str):
    """初始化資料庫，建立 addresses 表格"""
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS addresses (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        address TEXT UNIQUE,
        address_name TEXT,
        platform TEXT,
        symbol TEXT,
        created_at TEXT,
        last_balance TEXT DEFAULT '0'
    )
    """)
    conn.commit()
    return conn

def insert_address(conn: sqlite3.Connection, address: str, name: str, platform: str, symbol: str):
    """插入新地址到資料庫"""
    c = conn.cursor()
    now = datetime.utcnow().isoformat()
    try:
        c.execute("INSERT INTO addresses (address, address_name, platform, symbol, created_at) VALUES (?,?,?,?,?)",
                  (address, name, platform, symbol, now))
        conn.commit()
    except sqlite3.IntegrityError:
        # 地址已存在，跳過
        pass

def list_addresses(conn: sqlite3.Connection) -> List[Dict]:
    """列出資料庫中所有地址"""
    c = conn.cursor()
    c.execute("SELECT id, address, address_name, platform, symbol, created_at, last_balance FROM addresses")
    rows = c.fetchall()
    return [
        {"id": r[0], "address": r[1], "address_name": r[2], "platform": r[3], "symbol": r[4], "created_at": r[5], "last_balance": r[6]}
        for r in rows
    ]

def update_balance(conn: sqlite3.Connection, address: str, balance: str):
    """更新地址的餘額"""
    c = conn.cursor()
    c.execute("UPDATE addresses SET last_balance=? WHERE address=?", (str(balance), address))
    conn.commit()

print("✅ 資料庫函數載入完成！")


✅ 資料庫函數載入完成！


In [5]:
#### RIGSEC API 函數（真實模式）####

def get_access_token_real(api_base: str, api_key: str, client_id: str, client_secret: str, username: str):
    """
    取得 RIGSEC API 的存取權杖
    認證方式：signature = sha256(apikey + timestamp), Basic header with clientID:clientSecret
    """
    timestamp = str(int(time.time()))
    signature = hashlib.sha256((api_key + timestamp).encode()).hexdigest()
    basic = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {"Authorization": "Basic " + basic}
    data = {
        "grant_type": "api_user",
        "timestamp": timestamp,
        "username": username,
        "signature": signature
    }
    url = f"{api_base}/auth/oauth/token"
    resp = requests.post(url, headers=headers, data=data, timeout=10)
    resp.raise_for_status()
    j = resp.json()
    return j.get("access_token")

def create_address_real(api_base: str, token: str, blockchain: str, wallet_id: str, address_name: str, symbol: str):
    """透過 RIGSEC API 建立新地址"""
    url = f"{api_base}/wallet/{blockchain}/{wallet_id}/{address_name}/address"
    headers = {"Authorization": f"bearer {token}"}
    data = {"symbol": symbol}
    resp = requests.post(url, headers=headers, data=data, timeout=10)
    resp.raise_for_status()
    return resp.json()

def get_address_balance_real(api_base: str, token: str, platform: str, symbol: str, address: str):
    """透過 RIGSEC API 查詢地址餘額"""
    url = f"{api_base}/system/address/balance"
    headers = {"Authorization": f"bearer {token}"}
    params = {"platform": platform, "symbol": symbol, "address": address}
    resp = requests.get(url, headers=headers, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

print("✅ RIGSEC API 函數載入完成！")


✅ RIGSEC API 函數載入完成！


In [6]:
#### 模擬函數（測試模式）####

def mock_generate_address(name: str) -> str:
    """生成假的以太坊地址（用於測試）"""
    suffix = ''.join(random.choices('0123456789abcdef', k=38))
    return "0x" + suffix

def mock_get_balance_for_address(addr: str) -> str:
    """返回隨機的假餘額（用於測試）"""
    return str(round(random.uniform(0, 2.0), 6))

print("✅ 模擬函數載入完成！")


✅ 模擬函數載入完成！


In [7]:
#### 主要功能函數 ####

def bulk_generate_addresses(conn: sqlite3.Connection, n: int = 5):
    """批量生成地址"""
    mode = CONFIG["MODE"]
    if mode == "real":
        token = get_access_token_real(CONFIG["API_BASE"], CONFIG["API_KEY"],
                                      CONFIG["CLIENT_ID"], CONFIG["CLIENT_SECRET"], CONFIG["API_USERNAME"])
    else:
        token = None

    created = []
    for i in range(n):
        name = f"user_{int(time.time()) % 100000}_{i}"
        if mode == "real":
            resp = create_address_real(CONFIG["API_BASE"], token, CONFIG["BLOCKCHAIN"], CONFIG["WALLET_ID"], name, CONFIG["SYMBOL"])
            # 從 API 回應中解析地址
            addr = None
            try:
                addr = resp.get("data", {}).get("address")
            except Exception:
                addr = None
            if not addr:
                # 備用方案：如果格式不同
                addr = resp.get("data", {}).get("legacyAddress") if resp.get("data") else None
            if not addr:
                raise RuntimeError(f"無法從 API 回應中解析地址: {resp}")
        else:
            addr = mock_generate_address(name)

        insert_address(conn, addr, name, CONFIG["BLOCKCHAIN"], CONFIG["SYMBOL"])
        created.append((addr, name))
        print(f"[+] 已建立 {addr} 作為 {name}")

    return created

def refresh_all_balances(conn: sqlite3.Connection):
    """刷新所有地址的餘額"""
    mode = CONFIG["MODE"]
    token = None
    if mode == "real":
        token = get_access_token_real(CONFIG["API_BASE"], CONFIG["API_KEY"],
                                      CONFIG["CLIENT_ID"], CONFIG["CLIENT_SECRET"], CONFIG["API_USERNAME"])

    addrs = list_addresses(conn)
    results = []
    for a in addrs:
        if mode == "real":
            try:
                resp = get_address_balance_real(CONFIG["API_BASE"], token, a["platform"], a["symbol"], a["address"])
                # RIGSEC 的 system/address/balance 返回結構為 data.balance
                balance = None
                try:
                    balance = resp.get("data", {}).get("balance")
                except Exception:
                    balance = None
                if balance is None:
                    # 備用方案：如果 data 是字串/數字
                    balance = resp.get("data")
            except Exception as e:
                print(f"[!] 查詢 {a['address']} 餘額時發生錯誤: {e}")
                balance = "0"
        else:
            balance = mock_get_balance_for_address(a["address"])

        update_balance(conn, a["address"], str(balance))
        print(f"[=] {a['address']} 餘額 -> {balance}")
        results.append((a["address"], balance))
    return results

def poll_for_deposits(conn: sqlite3.Connection, rounds: int = 3, interval: int = 10):
    """
    輪詢監控存款
    每 `interval` 秒刷新餘額並偵測增加
    rounds: 輪詢次數（設為 None 則無限輪詢）
    """
    prev = {r["address"]: float(r["last_balance"]) if r["last_balance"] else 0.0 for r in list_addresses(conn)}
    iter_count = 0
    while True:
        iter_count += 1
        print(f"\n--- 輪詢第 {iter_count} 次 ({datetime.utcnow().isoformat()}) ---")
        refreshed = refresh_all_balances(conn)
        # 檢查餘額是否增加
        for addr, bal in refreshed:
            try:
                balf = float(bal)
            except:
                balf = 0.0
            prev_bal = prev.get(addr, 0.0)
            if balf > prev_bal + 1e-9:
                print(f"[存款] 地址 {addr} 餘額從 {prev_bal} 增加到 {balf}")
                # 這裡可以：記錄交易 ID、更新用戶帳戶、發送通知等
            prev[addr] = balf

        if rounds is not None and iter_count >= rounds:
            print("[*] 輪詢完成（已達到指定次數）")
            break

        time.sleep(interval)

print("✅ 主要功能函數載入完成！")


✅ 主要功能函數載入完成！


In [8]:
# 初始化資料庫
conn = init_db(CONFIG["DB_FILE"])
print(f"✅ 資料庫初始化完成: {CONFIG['DB_FILE']}")
print(f"✅ 模式: {CONFIG['MODE']}")


✅ 資料庫初始化完成: addresses.db
✅ 模式: mock


## 區塊 9: 批量生成地址

執行這個區塊來生成多個 ETH 地址（預設 5 個）

In [9]:
# 批量生成地址（可以修改 n 參數來改變數量）
created = bulk_generate_addresses(conn, n=5)
print(f"\n✅ 已生成 {len(created)} 個地址")


[+] 已建立 0xef1eec316c71c254005abd626430dbe4f63607 作為 user_58113_0
[+] 已建立 0x3ca48effe0b5de737f6c8846b0f1f1c92ba7cf 作為 user_58113_1
[+] 已建立 0x63077ec3699ea4d843b26f115a1c5a593f489f 作為 user_58113_2
[+] 已建立 0x2e3687d426da8e40f03f552e7710156eddc5c5 作為 user_58113_3
[+] 已建立 0xa651dda798e6be0d3269f474446dab7d2f66d3 作為 user_58113_4

✅ 已生成 5 個地址


/tmp/ipython-input-3344501027.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()


In [10]:
# 列出所有地址
addresses = list_addresses(conn)
print(f"\n📋 資料庫中共有 {len(addresses)} 個地址:\n")
for r in addresses:
    print(f"  - 地址: {r['address']}")
    print(f"    名稱: {r['address_name']}")
    print(f"    建立時間: {r['created_at']}")
    print(f"    最後餘額: {r['last_balance']}")
    print()



📋 資料庫中共有 5 個地址:

  - 地址: 0xef1eec316c71c254005abd626430dbe4f63607
    名稱: user_58113_0
    建立時間: 2025-11-10T07:01:53.254667
    最後餘額: 0

  - 地址: 0x3ca48effe0b5de737f6c8846b0f1f1c92ba7cf
    名稱: user_58113_1
    建立時間: 2025-11-10T07:01:53.264332
    最後餘額: 0

  - 地址: 0x63077ec3699ea4d843b26f115a1c5a593f489f
    名稱: user_58113_2
    建立時間: 2025-11-10T07:01:53.270674
    最後餘額: 0

  - 地址: 0x2e3687d426da8e40f03f552e7710156eddc5c5
    名稱: user_58113_3
    建立時間: 2025-11-10T07:01:53.276972
    最後餘額: 0

  - 地址: 0xa651dda798e6be0d3269f474446dab7d2f66d3
    名稱: user_58113_4
    建立時間: 2025-11-10T07:01:53.284174
    最後餘額: 0



In [11]:
# 刷新所有地址的餘額
print("🔄 正在刷新所有地址的餘額...")
results = refresh_all_balances(conn)
print(f"\n✅ 已更新 {len(results)} 個地址的餘額")


🔄 正在刷新所有地址的餘額...
[=] 0xef1eec316c71c254005abd626430dbe4f63607 餘額 -> 1.313602
[=] 0x3ca48effe0b5de737f6c8846b0f1f1c92ba7cf 餘額 -> 1.284009
[=] 0x63077ec3699ea4d843b26f115a1c5a593f489f 餘額 -> 0.664327
[=] 0x2e3687d426da8e40f03f552e7710156eddc5c5 餘額 -> 0.355167
[=] 0xa651dda798e6be0d3269f474446dab7d2f66d3 餘額 -> 1.38435

✅ 已更新 5 個地址的餘額


## 區塊 12: 開始監控存款（輪詢）

執行這個區塊來開始監控存款。程式會定期檢查餘額變化並偵測存款。

**注意：** 可以修改 `rounds` 參數來改變輪詢次數（設為 `None` 則無限輪詢）


In [12]:
# 開始監控存款
# rounds: 輪詢次數（設為 None 則無限輪詢）
# interval: 每次檢查的間隔時間（秒）
poll_for_deposits(conn, rounds=3, interval=CONFIG["POLL_INTERVAL"])


/tmp/ipython-input-3896835367.py:81: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"\n--- 輪詢第 {iter_count} 次 ({datetime.utcnow().isoformat()}) ---")



--- 輪詢第 1 次 (2025-11-10T07:02:33.949600) ---
[=] 0xef1eec316c71c254005abd626430dbe4f63607 餘額 -> 0.406837
[=] 0x3ca48effe0b5de737f6c8846b0f1f1c92ba7cf 餘額 -> 0.36511
[=] 0x63077ec3699ea4d843b26f115a1c5a593f489f 餘額 -> 1.240393
[=] 0x2e3687d426da8e40f03f552e7710156eddc5c5 餘額 -> 0.484391
[=] 0xa651dda798e6be0d3269f474446dab7d2f66d3 餘額 -> 1.757155
[存款] 地址 0x63077ec3699ea4d843b26f115a1c5a593f489f 餘額從 0.664327 增加到 1.240393
[存款] 地址 0x2e3687d426da8e40f03f552e7710156eddc5c5 餘額從 0.355167 增加到 0.484391
[存款] 地址 0xa651dda798e6be0d3269f474446dab7d2f66d3 餘額從 1.38435 增加到 1.757155

--- 輪詢第 2 次 (2025-11-10T07:02:43.985825) ---
[=] 0xef1eec316c71c254005abd626430dbe4f63607 餘額 -> 1.113584
[=] 0x3ca48effe0b5de737f6c8846b0f1f1c92ba7cf 餘額 -> 0.693761
[=] 0x63077ec3699ea4d843b26f115a1c5a593f489f 餘額 -> 1.137943
[=] 0x2e3687d426da8e40f03f552e7710156eddc5c5 餘額 -> 1.669365
[=] 0xa651dda798e6be0d3269f474446dab7d2f66d3 餘額 -> 0.423834
[存款] 地址 0xef1eec316c71c254005abd626430dbe4f63607 餘額從 0.406837 增加到 1.113584
[存款] 地址 0x

## 區塊 13: 下載資料庫檔案（可選）

如果你想下載資料庫檔案到本地，執行這個區塊


In [13]:
# 下載資料庫檔案
from google.colab import files

if os.path.exists(CONFIG["DB_FILE"]):
    files.download(CONFIG["DB_FILE"])
    print(f"✅ 已下載 {CONFIG['DB_FILE']}")
else:
    print(f"❌ 找不到檔案 {CONFIG['DB_FILE']}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 已下載 addresses.db
